In [1]:
import json
from autoprotocol.protocol import Protocol
from autoprotocol.instruction import Dispense
from autoprotocol.liquid_handle import Transfer
from autoprotocol.instruction import LiquidHandle
'''
Coding challenge for transcriptic/stemonix interview
written by Spencer Trinh

'''


solnVols_dict = {#adjusted volumes for std rxn soln (adjust vols 5x)
        "water": str(1.78*5) + ":mL",
        "RxnBuffer_20X": str(0.1*5) + ":uL",
        "DTT": str(0.02*5) + ":uL",
        "D_lucif": str(0.1*5) + ":uL",
        "firefly_lucif": str(0.0005*5) + ":uL"
		}

p = Protocol()

#=========================================================================================================
######REAGENT PREPARATION######
#=========================================================================================================

#1x reaction buffer
rxnBuffer_1X_tube = p.ref("1X_Rxn_Buffer",id=None,cont_type="micro-1.5",discard=True)
resID_20XrxnBuffer = "resID_20XrxnBuffer" #couldn't access the catalogue, assuming there is a resource_id for this reagent
p.dispense(rxnBuffer_1X_tube,
            "water",
            Dispense.builders.columns(
                [Dispense.builders.column(0,"950:uL")]
                )
            )
p.provision(resID_20XrxnBuffer,
        rxnBuffer_1X_tube.well(0),
        "50:uL"
        )
mixVol = 0.6*1000
p.mix(rxnBuffer_1X_tube.well(0),
        f"{mixVol}:uL"
    )

#=========================================================================================================
#generate 10mM D-luciferin stock / 3mg lyohphilised powder 
dLuciferinStock_tube = p.ref("DLucif_stock",id=None,cont_type="micro-1.5",storage="cold_20") #assuming 3mg of powder will be weighed into an eppi tube prior to 
p.transfer(source=rxnBuffer_1X_tube.well(0),
        destination=dLuciferinStock_tube.well(0),
        volume="1.0:mL"
        )
p.mix(dLuciferinStock_tube.well(0),
        f"{mixVol}:uL"
        )
p.agitate(ref=dLuciferinStock_tube,
          mode="vortex",
          speed="400:rpm",
          duration="1:minute"
          )
p.incubate(ref=dLuciferinStock_tube, #store away from light
        where="ambient",
        duration="15:minutes"
        )

#=========================================================================================================
#generate 100mM DTT stock / 25mg solid
#assuming that the 25 mg will be manually weighed into an eppi tube prior 
DTT_Stock_tube = p.ref("DTT_Stock_tube",id="dtt_stock_id",cont_type="micro-2.0",discard=True)
p.dispense(DTT_Stock_tube,
            "water",
            Dispense.builders.columns(
                [Dispense.builders.column(0,"1.62:mL")]
                )
            )
p.agitate(ref=DTT_Stock_tube,
          mode="vortex",
          speed="750:rpm",
          duration="1:minute"
          )
dtt_aliquot_dict = {f"dtt_aliquot_{i}" : p.ref(f"DTT_Alq_{i}",id=f"dtt_stock_id{i}",cont_type="micro-1.5",storage="cold_20") for i in range(2,11) }
dtt_aliquot_dict['dtt_aliquot_1'] = p.ref("DTT_Alq_1",id="dtt_stock_id1",cont_type="micro-1.5",discard=True) #add one manually to begin use

for i in range(2,11): #aliquot into 9/10 tubes and store into -20
    p.transfer(DTT_Stock_tube.well(0),
            dtt_aliquot_dict[f'dtt_aliquot_{i}'].well(0),
            "160:ul",
            one_tip=True )

p.transfer(DTT_Stock_tube.well(0), #aliquot one for use 
            dtt_aliquot_dict['dtt_aliquot_1'].well(0),
            "160:ul",
            one_tip=True )

#=========================================================================================================
#make 1uM ATP std 
ATP_startSoln_list = [p.ref(f"atp_startSoln_{i}",id=f"atp_startSoln_id{i}",cont_type="micro-1.5",discard=True) for i in range(1,5)]
resID_ATP_StockSoln = "resID_ATP_StockSoln" #substitute correct resource id from catalogue for this reagent 
diluVol = 10.0

for i in range(4):
    if i == 0:
        p.dispense(ATP_startSoln_list[i],
                    "water",
                    Dispense.builders.columns(
                        [Dispense.builders.column(0,"90:uL") ]
                        )
                    )
        p.provision(resource_id=resID_ATP_StockSoln, #aspirate from 5mM stock; results in 0.5mM
                    dests=ATP_startSoln_list[i].well(0),
                    volumes=f"{diluVol}:uL"
                   )
        p.agitate(ref=ATP_startSoln_list[i],
                  mode="vortex",
                  speed="750:rpm",
                  duration="1:minute"
                  )
    else: #1/10X dilution for first 3 dilutions, last one will be 1/5X
        if i == 3:
            diluVol = diluVol/2
        p.dispense(ATP_startSoln_list[i],
                    "water",
                    Dispense.builders.columns(
                        [Dispense.builders.column(0,"90:uL") ]
                        )
                    )
        p.transfer(source=ATP_startSoln_list[i-1].well(0),
                   destination=ATP_startSoln_list[i].well(0),
                   volume=f"{diluVol}:uL"
                  )
        p.agitate(ref=ATP_startSoln_list[i],
                  mode="vortex",
                  speed="750:rpm",
                  duration="1:minute"
                  )

#=========================================================================================================
#make 8 point 2-fold serial dilution of 1um ATP standard solution
ATP_8pt_stdDilution_plate = p.ref("ATP_8pt_stdDilution",id=None,cont_type="96-flat",discard=True)
p.transfer(source=ATP_startSoln_list[3].well(0), #1uM ATP (first xfer)
      destination=ATP_8pt_stdDilution_plate.well(0),
      volume="150:uL"
      )

for i in range(1,8): #2-fold dilution for each subsequent well
    p.provision("water", #not sure how to dispense just water into specific well
                ATP_8pt_stdDilution_plate.well(i),
                "50:uL"
             )
    p.transfer(source=ATP_8pt_stdDilution_plate.well(i-1),
              destination=ATP_8pt_stdDilution_plate.well(i),
              volume="50:uL",
              one_tip=True,
              method=Transfer( mix_after=True,
                               blowout=True
                             )
              )

#========================================================================================================= 
######STANDARD REACTION SOLUTION#######
#========================================================================================================= 
stdRxnSoln_24DeepPlate = p.ref("std_rxn_soln",id=None, cont_type="24-deep",storage="cold_20") #used this lw because has 10000ul max vol, aiming for 10ml
#add DI water
p.provision("water",
            stdRxnSoln_24DeepPlate.well(0),
            volumes=solnVols_dict['water']
            )
#add 20X rxn buffer
p.provision(resID_20XrxnBuffer,
            stdRxnSoln_24DeepPlate.well(0),
            solnVols_dict['RxnBuffer_20X']
            )
#add DTT
p.transfer(source=dtt_aliquot_dict['dtt_aliquot_1'].well(0),
            destination=stdRxnSoln_24DeepPlate.well(0),
            volume=solnVols_dict['DTT']
            )
#add d-luciferin
p.transfer(source=dLuciferinStock_tube.well(0),
            destination=stdRxnSoln_24DeepPlate.well(0),
            volume=solnVols_dict['D_lucif']
            )
#add firefly luciferase
resID_firefly = "resID_firefly" #substitute correct resource id from catalogue for this reagent 
firefly_lucif_tube = p.ref("firefly",id=None,cont_type="micro-1.5",storage="cold_20")
p.provision(resID_firefly,
        stdRxnSoln_24DeepPlate.well(0),
        solnVols_dict['firefly_lucif'],
        )
#mix std rxn soln
p.agitate(ref=stdRxnSoln_24DeepPlate,
          mode="vortex",
          speed="200:rpm",
          duration="1:minute"
          )
p.seal(stdRxnSoln_24DeepPlate,type="foil")
p.incubate(ref=stdRxnSoln_24DeepPlate, #store away from light
        where="ambient",
        duration="5:minutes"
        )

#========================================================================================================= 
######STANDARD CURVES & SAMPLES#######
#========================================================================================================= 
#aliquot from deep-24 std rxn plate to 96 deep well for easy transfer 
stdRxnSoln_96DWplate = p.ref("stdRxnSoln_96DWplate",id=None,cont_type="96-deep",discard=True)
p.unseal(stdRxnSoln_24DeepPlate) #remove seal prior to aspirating from
for j in range(8):
    p.transfer(source=stdRxnSoln_24DeepPlate.well(0),
            destination=stdRxnSoln_24DeepPlate.well(j),
            volume="1250:uL" #1250*8 = 10000ul
            )
    
#pipette 100ul of std rxn soln into 96well
read_plate = p.ref("read_plate",id=None, cont_type="96-flat-uv",discard=True) 
for j in range(12):    
    start = j*8
    end = (j*8)+8
    if j == 11:
        end = end -1 
    p.transfer(source=stdRxnSoln_96DWplate.wells(0,8),
                destination=read_plate.wells(start,end),
                volume="100:ul",
                method=Transfer(
                mix_before=True,
                dispense_z=LiquidHandle.builders.position_z(
                    reference="well_top"
                    )
                )
            )

#dispense just water for first colm (blank)
p.dispense(read_plate,
            "water",
            Dispense.builders.columns(
                [Dispense.builders.column(0,"10:uL")]
                )
            )

#dispense 10ul of ATP std soln (3 colms for triplcate = A2:H2 -> A4:H4)
for j in range(1,4):
    start = j*8
    end = (j*8)+8
    if j == 11:
        end = end -1 
    p.transfer(source=ATP_8pt_stdDilution_plate.wells(0,8),
            destination=read_plate.wells(start,end),
            volume="10:ul",
            method=Transfer( mix_after=True,
                             blowout=True
                             )
            )
    
#========================================================================================================= 
#assume 20 samples dispensed into 96 plate beforehand as mentioned in the protocol | dispense in triplcate into read plate and mix
sample_plate = p.ref("sample_plate",id=None, cont_type="96-flat",discard=True) 
grpNumOfWells = 8 #8 wells per colm
startWell=32 #starting on 5th column since first 4 are blanks and ATP stds soln
for j in range(8):
    if j >= 6:
        grpNumOfWells = 4 #4 wells per colm for last two columns (11 & 12)
    p.transfer(source=sample_plate.wells(j*grpNumOfWells,(j*grpNumOfWells)+grpNumOfWells),
            destination=read_plate.wells(startWell,startWell+grpNumOfWells),
            volume="10:ul",
            method=Transfer( mix_after=True,
                             blowout=True
                             )
            )
    startWell+=grpNumOfWells

#read plate
p.luminescence(ref=read_plate,
            wells=read_plate.all_wells(),
            dataref="read_plate"
            )

#========================================================================================================= 
#print(json.dumps(p.as_dict(),indent=2))
with open('/Users/spencertrinh/Downloads/coding_challenge_autoencode_jsondump.json','w') as f:
    json.dump(p.as_dict(),f,indent=2)

Instruction(incubate, {'object': Container(std_rxn_soln, cover=foil), 'where': 'ambient', 'duration': '5:minutes', 'shaking': False, 'co2_percent': 0})